# **Data Acquisition**

**Setting up environment**

In [ ]:
# Set your API key
%env OPENELECTRICITY_API_KEY=oe_3ZPqjof54kKpHxcpFqkpmYht

env: OPENELECTRICITY_API_KEY=oe_3ZPqjof54kKpHxcpFqkpmYht


In [ ]:
import os, time, math, requests
from datetime import datetime, timezone
import os, csv, requests
from collections import defaultdict

## **Retrieve all facility code**

**API call to get all unique facility and unit code**

In [ ]:
API = "https://api.openelectricity.org.au"
TOKEN = os.environ.get("OPENELECTRICITY_API_KEY") or "oe_3ZVkmJvbjLU4yPJCHaZXhBgS"
H = {"Authorization": f"Bearer {TOKEN}", "Accept":"application/json"}

def get(url, params):
    r = requests.get(url, headers=H, params=params, timeout=60)
    r.raise_for_status()
    return r.json()


# Power + emissions by region
resp2 = get(f"{API}/v4/facilities/", {
    'network_id': 'NEM',
    'status_id': 'operating'
})

**flatten the retrieved data to a normalised pandas dataframe**

In [ ]:
import pandas as pd

# Flatten to "one row per UNIT", while carrying facility metadata down via `meta`
units_flat = pd.json_normalize(
    resp2["data"],
    record_path="units",
    meta=[
        "code", "name", "network_id", "network_region",
        "description", "updated_at", "created_at",
        ["location", "lat"], ["location", "lng"]
    ],
    meta_prefix="facility_",       # avoid name clashes with unit fields
    errors="ignore"
)

# Rename columns
facilities_df = (
    units_flat
      .rename(columns={
          "code": "unit_code",                  # unit code -> subfacility
          "dispatch_type": "dispatch_type",
          "facility_code": "facility_code",
          "facility_name": "name",
          "facility_network_id": "network_id",
          "facility_network_region": "network_region",
          "facility_description": "description",
          "facility_updated_at": "updated_at",
          "facility_created_at": "created_at",
          "facility_location.lat": "lat",
          "facility_location.lng": "lng",
      })
      # Reorder
      .loc[:, [
          "facility_code", "unit_code", "dispatch_type", "fueltech_id",
          "name", "network_id", "network_region", "lat", "lng"
      ]]
)

# make lat/lng numeric
facilities_df["lat"] = pd.to_numeric(facilities_df["lat"], errors="coerce")
facilities_df["lng"] = pd.to_numeric(facilities_df["lng"], errors="coerce")

facilities_df


,facility_code,unit_code,dispatch_type,fueltech_id,name,network_id,network_region,lat,lng
0,ADP,ADPBA1,BIDIRECTIONAL,battery,Adelaide Desalination,NEM,SA1,-35.096948,138.484061
1,ADP,ADPBA1G,GENERATOR,battery_discharging,Adelaide Desalination,NEM,SA1,-35.096948,138.484061
2,ADP,ADPBA1L,LOAD,battery_charging,Adelaide Desalination,NEM,SA1,-35.096948,138.484061
3,ADP,ADPPV1,GENERATOR,solar_utility,Adelaide Desalination,NEM,SA1,-35.096948,138.484061
4,ADP,ADPPV2,GENERATOR,solar_utility,Adelaide Desalination,NEM,SA1,-35.096948,138.484061
...,...,...,...,...,...,...,...,...,...
639,YWNGAHYD,YWNGAHYD,GENERATOR,hydro,Yarrawonga,NEM,VIC1,-36.009466,145.999568
640,YARWUN,YARWUN_1,GENERATOR,gas_ccgt,Yarwun,NEM,QLD1,-23.830200,151.149692
641,YATSF1,YATSF1,GENERATOR,solar_utility,Yatpool,NEM,VIC1,-34.380730,142.205340
642,YAWWF,YAWWF1,GENERATOR,wind,Yawong,NEM,VIC1,-36.471022,143.361722


In [ ]:
#store facility code in an array
unique_facility_codes = facilities_df['facility_code'].unique()
unique_facility_codes

array(['ADP', 'ALDGASF', 'ANGASTON', 'APPIN', 'ARWF', 'AVLSF', 'DEIBDL',
       'BAKING', 'BHWF', 'BALBESS', 'BANGOWF', 'BAPS', 'BANNSP',
       'BARCALDN', 'BARCSF', 'BARKIPS', 'BARRON', 'BASTYAN', 'BAYSW',
       'BBP31', 'BRYB1WF1', 'BERWICK', 'BERYLSF', 'BIALAWF', 'BLAYNEY',
       'SNOWY6', 'BLUEGSF', 'BLYTHB', 'BOCOROCK', 'BODWF', 'MCKAY',
       'BOLIVAR', 'BOLIVPS', 'BOMENSF', 'BBATTERY', 'BRAEMARA', 'B2PS',
       'BRNDBES', 'BWTR1', 'BHILLGT', 'BROKENH', 'BHB', 'BROOKLYN',
       'BROWNMT', 'BPLANDF', 'BULGANA', 'BNGSF1', 'BNGSF2', 'BDONGHYD',
       'BURRIN', 'BUTLERSG', 'CALL_B', 'CALLIDEC1', 'CANUNDA1', 'CAPBES',
       'CAPTL_WF', 'CESF', 'LI_WY_CA', 'CATHROCK', 'CTHLWF', 'CETHANA',
       'CHALLWF', 'CHPSTWF', 'CHYTWF', 'CHILDSF', 'CHBESS', 'CBWWBA',
       'CLARESF', 'CLRKCWF', 'CLAYTON', 'CLEMGPWF', 'CLERMSF', 'CLOVER',
       'CLUNY', 'CODRGTON', 'COHUNSF1', 'COLEASF', 'COLWF01', 'CSPVPS',
       'COLONGRA', 'COLUMSF', 'CPSA', 'CONDONG1', 'CBWF', 'COOPGWF',
       'CO

## **Retrieve data (metrics) for all facilities**

Now since we have all the facility code, we can make another series of API call to retrieve all the required metrics. But here, it gets tricky as we can only retrieve data for at most 10 facilities per time, else we get a "bad request" warning. So to tackle this, I will implement a loop that makes 50 API calls with a delay of 0.6 seconds between each call to respect the API :)))

### **Retrieving Facility metrics in batch (10 per batch)**

In [ ]:
import time
import pandas as pd

BATCH_SIZE = 10
SLEEP_SEC  = 0.6     # small pause between calls
DATE_START = "2025-10-07T00:00:00"  # NEM time, timezone-naive
DATE_END   = "2025-10-14T00:00:00"  # exclusive
STEP_HOURS = 5/60  # 5-minute interval
cells = {}

for i in range(0, len(unique_facility_codes), BATCH_SIZE):
    batch = unique_facility_codes[i:i+BATCH_SIZE]

    resp = get(f"{API}/v4/data/facilities/NEM", {
        "facility_code": batch,
        "metrics": ["power", "emissions"],
        "interval": "5m",
        "date_start": DATE_START,
        "date_end": DATE_END
    })

    for block in resp.get("data", []):
        metric = block.get("metric")  # "power" or "emissions"
        for series in block.get("results", []):
            cols = series.get("columns") or {}
            unit_code = cols.get("unit_code")

            if not unit_code:
                # fallback: try parse from name like "power_MP1"
                name = series.get("name", "")
                unit_code = name.split("_")[-1] if "_" in name else None
            if not unit_code:
                continue

            for ts, val in series.get("data", []):
                key = (unit_code, ts)
                row = cells.setdefault(key, {"unit_code": unit_code, "ts": ts})
                row[metric] = val

    time.sleep(SLEEP_SEC)  # be polite to the API

# build the single dataframe for all 500 facilities
df = pd.DataFrame(list(cells.values()))

# dtypes
df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")
for c in ["power", "emissions"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# add facility_code from units_flat mapping
unit_to_facility = dict(zip(units_flat["code"], units_flat["facility_code"]))
df["facility_code"] = df["unit_code"].map(unit_to_facility)

# tidy columns
want_cols = ["facility_code", "unit_code", "ts", "power", "emissions"]
for col in want_cols:
    if col not in df.columns:
        df[col] = pd.NA

df = (
    df[want_cols]
    .sort_values(["facility_code", "unit_code", "ts"])
    .reset_index(drop=True)
)

print(df.shape)
df.head()


(985459, 5)


,facility_code,unit_code,ts,power,emissions
0,0MREH,MREHA1,2025-10-06 14:00:00+00:00,-3.3805,0.0
1,0MREH,MREHA1,2025-10-06 14:05:00+00:00,0.9567,0.0
2,0MREH,MREHA1,2025-10-06 14:10:00+00:00,-1.9638,0.0
3,0MREH,MREHA1,2025-10-06 14:15:00+00:00,0.5740,0.0
4,0MREH,MREHA1,2025-10-06 14:20:00+00:00,-1.9504,0.0


**Save to csv files to avoid susequent APi call**

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("facilities_metrics.csv", index=False)

facilities_df.to_csv("facilities_data.csv", index=False)


## **Now we import the data and perform some cleaning**

In [ ]:
import pandas as pd

# Import the CSV files into DataFrames
df = pd.read_csv("facilities_metrics.csv")
facilities_df = pd.read_csv("facilities_data.csv")


**Merge facility metric and facility data**

In [ ]:
# Merge df and facilities_df on facility_code and unit_code
merged_df = pd.merge(df, facilities_df, on=['facility_code', 'unit_code'], how='left')

print("Merged DataFrame:")
merged_df.head()

Merged DataFrame:


,facility_code,unit_code,ts,power,emissions,dispatch_type,fueltech_id,name,network_id,network_region,lat,lng
0,0MREH,MREHA1,2025-10-06 14:00:00+00:00,-3.3805,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
1,0MREH,MREHA1,2025-10-06 14:05:00+00:00,0.9567,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
2,0MREH,MREHA1,2025-10-06 14:10:00+00:00,-1.9638,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
3,0MREH,MREHA1,2025-10-06 14:15:00+00:00,0.5740,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
4,0MREH,MREHA1,2025-10-06 14:20:00+00:00,-1.9504,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302


In [ ]:
merged_df

,facility_code,unit_code,ts,power,emissions,dispatch_type,fueltech_id,name,network_id,network_region,lat,lng
0,0MREH,MREHA1,2025-10-06 14:00:00+00:00,-3.3805,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
1,0MREH,MREHA1,2025-10-06 14:05:00+00:00,0.9567,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
2,0MREH,MREHA1,2025-10-06 14:10:00+00:00,-1.9638,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
3,0MREH,MREHA1,2025-10-06 14:15:00+00:00,0.5740,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
4,0MREH,MREHA1,2025-10-06 14:20:00+00:00,-1.9504,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
...,...,...,...,...,...,...,...,...,...,...,...,...
989486,NaN,HUNTER2,2025-10-13 13:35:00+00:00,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989487,NaN,HUNTER2,2025-10-13 13:40:00+00:00,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989488,NaN,HUNTER2,2025-10-13 13:45:00+00:00,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989489,NaN,HUNTER2,2025-10-13 13:50:00+00:00,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df.isna().sum()

,0
facility_code,2016
unit_code,0
ts,0
power,0
emissions,0
dispatch_type,2016
fueltech_id,2016
name,2016
network_id,2016
network_region,2016


In [ ]:
merged_df["facility_code"].isna().sum()

np.int64(2016)

*Notice: there are some missing value for facility_code, which is from facility HUNTER, because the api call to facility data only match facility HUNTER with unit HUNTER1, but the facility_metrics provides data for both HUNTER1 and HUNTER2 --> HUNTER2 fail to match*

**Handle**: we can safely drop all data rows for HUNTER, as their power and emission are all 0s, despite being classified as 'operating'

# **Data Cleaning**

In [ ]:
# Drop rows where facility_code is NA
merged_df = merged_df.dropna(subset=["facility_code", "unit_code"]).copy()

# consistent casing for keys (all uppercase)
merged_df["facility_code"] = merged_df["facility_code"].astype(str).str.upper()
merged_df["unit_code"]     = merged_df["unit_code"].astype(str).str.upper()


# Timestamps: parse to UTC
merged_df["ts"] = pd.to_datetime(merged_df["ts"], utc=True, errors="coerce")

# Ensure numerics for some attributes
for c in ["power", "emissions"]:
    if c in merged_df.columns:
        merged_df[c] = pd.to_numeric(merged_df[c], errors="coerce")


In [ ]:
merged_df["facility_code"].isna().sum()

np.int64(0)

**There are some facilties that are non-active for the entire period of time ( they have all zero entries for both power and emssion across all units for the entire timeframe) so we do not want to consider that**

In [ ]:
# Remove facilities that are all-zero at every timestamp
#    i.e., for a given facility_code, every row has power==0 AND emissions==0
all_zero_by_row = (merged_df["power"].fillna(0).eq(0) &
                   merged_df["emissions"].fillna(0).eq(0))

all_zero_facilities = all_zero_by_row.groupby(merged_df["facility_code"]).transform("all")

merged_df = merged_df[~all_zero_facilities].copy()
merged_df.reset_index(drop=True, inplace=True)


**check for duplicates**

In [ ]:
d = merged_df.duplicated(subset=["unit_code","ts"], keep=False)
dupes = merged_df[d].sort_values(["unit_code","ts"])
print(f"Duplicate key rows: {dupes.shape[0]} "
      f"(distinct problematic keys: {dupes[['unit_code','ts']].drop_duplicates().shape[0]})")


Duplicate key rows: 0 (distinct problematic keys: 0)


In [ ]:
merged_df

,facility_code,unit_code,ts,power,emissions,dispatch_type,fueltech_id,name,network_id,network_region,lat,lng
0,0MREH,MREHA1,2025-10-06 14:00:00+00:00,-3.3805,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
1,0MREH,MREHA1,2025-10-06 14:05:00+00:00,0.9567,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
2,0MREH,MREHA1,2025-10-06 14:10:00+00:00,-1.9638,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
3,0MREH,MREHA1,2025-10-06 14:15:00+00:00,0.5740,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
4,0MREH,MREHA1,2025-10-06 14:20:00+00:00,-1.9504,0.0,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
...,...,...,...,...,...,...,...,...,...,...,...,...
915997,YSWF,YSWF1,2025-10-13 13:35:00+00:00,0.0000,0.0,GENERATOR,wind,Yaloak South,NEM,VIC1,-37.716474,144.241947
915998,YSWF,YSWF1,2025-10-13 13:40:00+00:00,0.0000,0.0,GENERATOR,wind,Yaloak South,NEM,VIC1,-37.716474,144.241947
915999,YSWF,YSWF1,2025-10-13 13:45:00+00:00,0.0000,0.0,GENERATOR,wind,Yaloak South,NEM,VIC1,-37.716474,144.241947
916000,YSWF,YSWF1,2025-10-13 13:50:00+00:00,-0.1000,0.0,GENERATOR,wind,Yaloak South,NEM,VIC1,-37.716474,144.241947


**Shape the data (order by ts for later publishing)**

In [ ]:
merged_df = merged_df.sort_values(["ts", "facility_code"]).reset_index(drop=True)
merged_df

,facility_code,unit_code,ts,power,emissions,dispatch_type,fueltech_id,name,network_id,network_region,lat,lng
0,0MREH,MREHA1,2025-10-06 14:00:00+00:00,-3.3805,0.0000,BIDIRECTIONAL,battery,Melbourne A1,NEM,VIC1,-37.661274,144.726302
1,0MREH,MREHAL1,2025-10-06 14:00:00+00:00,3.3805,0.0000,LOAD,battery_charging,Melbourne A1,NEM,VIC1,-37.661274,144.726302
2,0MREHA2,MREHA2,2025-10-06 14:00:00+00:00,0.0000,0.0000,BIDIRECTIONAL,battery,Melbourne A2,NEM,VIC1,-37.663934,144.726927
3,0MREHA2,MREHAG2,2025-10-06 14:00:00+00:00,0.0000,0.0000,GENERATOR,battery_discharging,Melbourne A2,NEM,VIC1,-37.663934,144.726927
4,0WAMBOWF,WAMBOWF1,2025-10-06 14:00:00+00:00,48.5090,0.0000,GENERATOR,wind,Wambo,NEM,QLD1,-26.603045,151.246876
...,...,...,...,...,...,...,...,...,...,...,...,...
915997,YARANSF,YARANSF1,2025-10-13 13:55:00+00:00,0.0000,0.0000,GENERATOR,solar_utility,Yarranlea,NEM,QLD1,-27.708939,151.532696
915998,YARWUN,YARWUN_1,2025-10-13 13:55:00+00:00,87.5900,4.5247,GENERATOR,gas_ccgt,Yarwun,NEM,QLD1,-23.830200,151.149692
915999,YATSF1,YATSF1,2025-10-13 13:55:00+00:00,0.0000,0.0000,GENERATOR,solar_utility,Yatpool,NEM,VIC1,-34.380730,142.205340
916000,YENDONWF,YENDWF1,2025-10-13 13:55:00+00:00,3.2600,0.0000,GENERATOR,wind,Yendon,NEM,VIC1,-37.630952,144.022463


**Remove all the trailing 1s in network region**

In [ ]:
# Get unique values of the 'network_region'
unique_regions = merged_df['network_region'].unique()

print("Unique values in 'network_region':")
display(unique_regions)

Unique values in 'network_region':


array(['VIC1', 'QLD1', 'SA1', 'NSW1', 'TAS1'], dtype=object)

In [ ]:
# Remove trailing '1' from network_region
merged_df['network_region'] = merged_df['network_region'].astype(str).str.replace('1$', '', regex=True)

print("Network regions after removing trailing '1':")
display(merged_df['network_region'].unique())

Network regions after removing trailing '1':


array(['VIC', 'QLD', 'SA', 'NSW', 'TAS'], dtype=object)

In [ ]:
merged_df["network_region"].unique()

array(['VIC', 'QLD', 'SA', 'NSW', 'TAS'], dtype=object)

# **Data Publishing**

In [ ]:
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
import paho.mqtt.client as mqtt

**Setting up clients**

In [ ]:
import json, time
from paho.mqtt import client as mqtt
import math

# Broker info
BROKER_HOST = "broker.hivemq.com"
BROKER_PORT = 1883
CLIENT_ID    = "5339-ass2-facility-publisher"
BASE_TOPIC   = "facility"

# MQTT client
client = mqtt.Client(client_id=CLIENT_ID, clean_session=True)
client.reconnect_delay_set(min_delay=1, max_delay=30)

def on_connect(cli, userdata, flags, rc):
    print(f"Connected with result code {rc}")

def on_publish(cli, userdata, mid):
    pass

client.on_connect  = on_connect
client.on_publish  = on_publish

client.connect(BROKER_HOST, BROKER_PORT, keepalive=60)
client.loop_start()

/tmp/ipython-input-266698249.py:12: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=CLIENT_ID, clean_session=True)


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

**Start publishing**

The code below set up the MQTT client and start publishing

To target the 'only updated power/emissions values are published' issue, we use a hashmap to store previous facility units details and compare it to current details to see if there's any change, if there's no change, we will not publish it. Full detailed analysis in the report.

In [ ]:
REPLAY_COOLDOWN_SECS = 60  # wait before restarting when we hit the end

#  Some other settings
PUBLISH_QOS    = 1            # At most 1 semantic
RETAIN_LAST    = False
ROW_DELAY_SECS = 0.1



# Guarantee that "only updated power/emissions values are published" as spcified in ED FAQ
last_vals = {}      # this is a hashmap with key and value (facility_code, unit_code) -> (power, emissions)
def same(a, b):
    if a is None and b is None: return True
    if (a is None) ^ (b is None): return False
    return math.isclose(a, b, rel_tol=0.0, abs_tol=1e-6)

# Start the publishing loop
while True:
    # reset per-replay cache so first occurrences publish again
    last_vals.clear()

    for ts_value, batch in merged_df.groupby("ts", sort=True):
        print(f"\n Publishing batch for timestamp {ts_value}")

        for row in batch.itertuples(index=False):
            facility_code_cleaned = row.facility_code.replace('+','').replace('#','')
            unit_code_cleaned     = row.unit_code.replace('+','').replace('#','')

            # current values (coerce to floats/None)
            cur_power     = float(row.power)     if pd.notna(row.power)     else None
            cur_emissions = float(row.emissions) if pd.notna(row.emissions) else None

            key  = (row.facility_code, row.unit_code)
            prev = last_vals.get(key)

            # skip if unchanged since last publish (within this replay)
            if prev is not None and same(cur_power, prev[0]) and same(cur_emissions, prev[1]):
                continue

            topic = f"facility/{facility_code_cleaned}/{unit_code_cleaned}"
            payload = json.dumps({
                "facility_code": row.facility_code if pd.notna(row.facility_code) else None,
                "unit_code":     row.unit_code     if pd.notna(row.unit_code)     else None,
                "state":         row.network_region if pd.notna(row.network_region) else None,
                "lat":           float(row.lat)    if pd.notna(row.lat)    else None,
                "lng":           float(row.lng)    if pd.notna(row.lng)    else None,
                "fueltech_id":   row.fueltech_id   if pd.notna(row.fueltech_id)   else None,
                "ts":            ts_value.strftime("%Y-%m-%dT%H:%M:%SZ"),
                "power":         float(row.power)      if pd.notna(row.power)      else None,
                "emissions":     float(row.emissions)  if pd.notna(row.emissions)  else None,
                "dispatch_type": row.dispatch_type if pd.notna(row.dispatch_type) else None,
                "name":          row.name          if pd.notna(row.name)          else None,
                "network_id":    row.network_id    if pd.notna(row.network_id)    else None,
            })

            info = client.publish(topic, payload, qos=PUBLISH_QOS, retain=RETAIN_LAST)
            info.wait_for_publish()
            print(f"Published → {topic}: {payload}")

            last_vals[key] = (cur_power, cur_emissions)

        time.sleep(ROW_DELAY_SECS)

    print(f"\nReached end of dataset — waiting {REPLAY_COOLDOWN_SECS}s, then restarting replay.")
    time.sleep(REPLAY_COOLDOWN_SECS)


Streaming output truncated to the last 5000 lines.
Published → facility/YSWF/YSWF1: {"facility_code": "YSWF", "unit_code": "YSWF1", "state": "VIC", "lat": -37.716474, "lng": 144.241947, "fueltech_id": "wind", "ts": "2025-10-06T15:25:00Z", "power": 0.9, "emissions": 0.0, "dispatch_type": "GENERATOR", "name": "Yaloak South", "network_id": "NEM"}

 Publishing batch for timestamp 2025-10-06 15:30:00+00:00
Published → facility/0WAMBOWF/WAMBOWF1: {"facility_code": "0WAMBOWF", "unit_code": "WAMBOWF1", "state": "QLD", "lat": -26.603045, "lng": 151.246876, "fueltech_id": "wind", "ts": "2025-10-06T15:30:00Z", "power": 30.816, "emissions": 0.0, "dispatch_type": "GENERATOR", "name": "Wambo", "network_id": "NEM"}
Published → facility/ADP/ADPBA1: {"facility_code": "ADP", "unit_code": "ADPBA1", "state": "SA", "lat": -35.096948, "lng": 138.484061, "fueltech_id": "battery", "ts": "2025-10-06T15:30:00Z", "power": 0.088, "emissions": 0.0, "dispatch_type": "BIDIRECTIONAL", "name": "Adelaide Desalination",

KeyboardInterrupt: 

Stop and disconnect for subsequent pulishing

In [ ]:
client.loop_stop()
client.disconnect()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

# **Subscribing to topics**

Link to [Task 4](https://colab.research.google.com/drive/1po1VCTFsqmRZB_My2syF6yOzmx0wIMh5?usp=sharing)